In [21]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ratikkakkar/electric-vehicle-population-data")

print("Path to dataset files:", path)

Path to dataset files: /Users/alessandrogobbo/.cache/kagglehub/datasets/ratikkakkar/electric-vehicle-population-data/versions/1


In [22]:
import polars as pl
data = pl.read_csv(path+'/Electric_Vehicle_Population_Data.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

Elimino alcune colonne che non sono utili per l'analisi che sto eseguendo.

In [23]:
data = data.select(pl.exclude(['VIN (1-10)','Postal Code','Base MSRP','Legislative District','DOL Vehicle ID','Electric Utility','2020 Census Tract']))

In [ ]:
data.head()

Una volta eliminato le colonne che non risultavano utili al nostro fine, cominciamo ad eseguire delle analisi univariate.

In [ ]:
data.get_column('Clean Alternative Fuel Vehicle (CAFV) Eligibility').unique()

In [ ]:
print(list(data.columns))

A seguito di una veloce analisi noto che la colonna 'Clean Alternative Fuel Vehicle (CAFV) Eligibility' non mi è utile per questa analisi.

In [24]:
data = data.select(pl.exclude('Clean Alternative Fuel Vehicle (CAFV) Eligibility'))

In [ ]:
data.head()

Ora che ho selezionato le colonne che mi sono utili allo scopo, posso cominciare l'analisi univariata.
# ANALISI DELLE VARIABILI

In [ ]:
#Numero di vendite in base all'anno 
(
    data
    .group_by('Model Year')
    .agg(
        tot_per_year = pl.col('Model Year').sum()
    )
    .sort(pl.col('Model Year'), descending=False)
)

In [ ]:
#Numero di vendite in base all'anno e marca
(
    data
    .group_by('Model Year', 'Make')
    .agg(
        tot_per_year = pl.col('Model Year').sum()
    )
    .sort(pl.col('Model Year'))
)

In [ ]:
#Distribuzione del range di elettrico 
(
    data
    .group_by('Model Year', 'Make')
    .agg(
        max_range = pl.col('Electric Range').max(),
        min_range = pl.col('Electric Range').min(),
        mean_range = pl.col('Electric Range').mean()
    )
    .sort(pl.col('Model Year'))
)

Noto che alcune auto hanno valori nulli sul range di elettrico, quindi verranno succesivamente rimossi 
dal dataset quando si vorrà eseguire l'analisi dell'autonomia dell'elettrico.

In [ ]:
#Distribuzione del range di elettrico 
(
    data
    .group_by('Model Year', 'Make')
    .agg(
        max_range = pl.col('Electric Range').max(),
        min_range = pl.col('Electric Range').min(),
        mean_range = pl.col('Electric Range').mean()
    )
    .sort(pl.col('Model Year'))
)

Una volta eseguiti i vari controlli, si andrà a creare un file csv in cui si salveranno tutti i dati all'interno della cartella DATA

In [ ]:
(
    data
    .group_by('Model Year')
    .agg(
        tot_per_year = pl.col('Model Year').count()
    )
    .sort(pl.col('Model Year'))
)

In [ ]:
(
    data
    .group_by('Model Year')
    .agg(
        tot_per_year = pl.col('Model Year').count()
    )
    .filter(pl.col('Model Year')<=2010)
    .sort(pl.col('Model Year'))
)

In [ ]:
data = (
    data
    .filter(pl.col('Model Year')>2010)
)


In [ ]:
data.describe()

In [ ]:
#data.write_csv('../DATA/data.csv')

In [7]:
data.head()

Vehicle Location
str
"""POINT (-81.80023 24.5545)"""
"""POINT (-114.57245 35.16815)"""
"""POINT (-120.50721 46.60448)"""
"""POINT (-121.7515 48.53892)"""
"""POINT (-122.20596 47.97659)"""


In [25]:
data_test = (
        data.select(pl.col('Vehicle Location'), pl.col('Make'))
    )

In [26]:
data_test.head()

Vehicle Location,Make
str,str
"""POINT (-81.80023 24.5545)""","""TOYOTA"""
"""POINT (-114.57245 35.16815)""","""CHEVROLET"""
"""POINT (-120.50721 46.60448)""","""NISSAN"""
"""POINT (-121.7515 48.53892)""","""CHEVROLET"""
"""POINT (-122.20596 47.97659)""","""FORD"""


In [27]:
data_test = (
    data_test
    .filter(pl.col('Make') == 'TESLA')
)

In [ ]:
data_test.head()

Vehicle Location,Make
str,str
"""POINT (-122.18384 47.8031)""","""TESLA"""
"""POINT (-119.2771 47.13196)""","""TESLA"""
"""POINT (-122.33029 48.46846)""","""TESLA"""
"""POINT (-73.84643 43.1284)""","""TESLA"""
"""POINT (-122.31768 47.87166)""","""TESLA"""


In [33]:
data_test = data_test.select(pl.exclude('Make'))
data_test.head()

Vehicle Location
str
"""POINT (-122.18384 47.8031)"""
"""POINT (-119.2771 47.13196)"""
"""POINT (-122.33029 48.46846)"""
"""POINT (-73.84643 43.1284)"""
"""POINT (-122.31768 47.87166)"""


In [76]:
import re
import pandas as pd

coord_list = []
for row in data_test.rows():
    if row[0] is not None:
        numb = re.findall(r'-?\d+\.\d+|-?\d+', row[0])
        #print(numb)
        coord = (float(numb[0]), float(numb[1]))
        coord_list.append(coord)
        #print(row[0])
    
#print(coord_list)
#print(type(coord_list[0][0]))

coord = pd.DataFrame(coord_list, columns=['lon', 'lat'])
print(coord.dtypes)
print(coord.columns)
print(coord.head())

lon    float64
lat    float64
dtype: object
Index(['lon', 'lat'], dtype='object')
         lon       lat
0 -122.18384  47.80310
1 -119.27710  47.13196
2 -122.33029  48.46846
3  -73.84643  43.12840
4 -122.31768  47.87166


In [ ]:
import streamlit as st
import pydeck as pdk

coord_list = []

data = (
    data.select(pl.col('Vehicle Location'))
)

for row in data.rows():
    if row[0] is not None:
        numb = re.findall(r'\d+\.\d+|\d+', row[0])
        coord = (float(numb[0]), float(numb[1]))
        coord_list.append(coord)

coord_chart = pd.DataFrame(coord_list, columns=['lat', 'lon'])

st.pydeck_chart(
    pdk.Deck(
        map_style=None,
        initial_view_state=pdk.ViewState(
            latitude=122,
            longitude=47,
            zoom=11,
            pitch=50,
        ),
        layers=[
            pdk.Layer(
                "HexagonLayer",
                data=coord_chart,
                get_position="[lon, lat]",
                radius=200,
                elevation_scale=4,
                elevation_range=[0, 1000],
                pickable=True,
                extruded=True,
            ),
            pdk.Layer(
                "ScatterplotLayer",
                data=coord_chart,
                get_position="[lon, lat]",
                get_color="[200, 30, 0, 160]",
                get_radius=200,
            ),
        ],
    )
)

2024-12-04 16:14:36.243 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-04 16:14:37.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-04 16:14:37.289 
  command:

    streamlit run /Users/alessandrogobbo/Documents/PROJECT_SE2/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-04 16:14:37.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()